In [10]:
from pystac_client import Client
from odc.stac import load
import odc.geo
import xarray as xr
import rioxarray as rio
import geopandas as gpd 
import itertools
from multiprocessing import Pool
import dask.distributed
import dask.utils
from odc.stac import configure_rio, stac_load

from odc.algo import erase_bad, mask_cleanup

CLOUD_SHADOWS = 3
CLOUD_HIGH_PROBABILITY = 9

bitmask = 0
for field in [CLOUD_SHADOWS, CLOUD_HIGH_PROBABILITY]:
    bitmask |= 1 << field


# Start dask cluster
client = dask.distributed.Client()
configure_rio(cloud_defaults=True, client = client)

# Prepare region of interest
# spain = gpd.read_file('/Users/diegobengochea/git/iberian.carbon/data/SpainPolygon/gadm41_ESP_1.shp')
spain = gpd.read_file('/home/dibepa/git/iberian.carbon/spain/gadm41_ESP_1.shp')
# Filter continental Spain
spain = spain[ (spain.GID_1 != 'ESP.7_1') & (spain.GID_1 != 'ESP.13_1') & (spain.GID_1 != 'ESP.14_1') ] 
# Reproject to UTM30 EPSG:25830
spain = spain.dissolve()[['geometry','COUNTRY']].to_crs(epsg='25830')
# Add CRS information to shapely polygon
geometry_spain = odc.geo.geom.Geometry(spain.geometry[0],crs='EPSG:25830')

print('Here')

# Create a GeoBox for all continental Spain with a 10 meters resolution 
geobox_spain = odc.geo.geobox.GeoBox.from_geopolygon(geometry_spain,resolution=10) # The resolution here is irrelevant since the Spain GeoBOX is too large to make queries, adn thus cannot be used as intersect. Only used to create tiles of suitable shape 20km2

print('Here1')


# Divide the full geobox in Geotiles of smaller size for processing
geotiles_spain = odc.geo.geobox.GeoboxTiles(geobox_spain,(10000,10000))
# geotiles_spain = odc.geo.geobox.GeoboxTiles(geobox_spain,(100,100))
print('Here 2')

geotiles_spain = [ geotiles_spain.__getitem__(tile) for tile in geotiles_spain._all_tiles() ][0]

print('Here  3')


/home/dibepa/.conda/envs/sentinel.mosaics/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39033 instead
  warnings.warn(


Here
Here1
Here 2
Here  3


In [12]:
print('In here')

geobox = geotiles_spain
year = 2018

bbox = geobox.boundingbox.to_crs('EPSG:4326')
bbox = (bbox.left,bbox.bottom,bbox.right,bbox.top)

green_season = f'{year}-04-01/{year}-10-01'

catalog = Client.open("https://earth-search.aws.element84.com/v1") 
search = catalog.search(
    collections = ['sentinel-2-l2a'],
    bbox = bbox, 
    datetime = green_season,
    query = ['eo:cloud_cover<50']
)

item_collection = search.item_collection()
print(len(item_collection))

src_dataset = odc.stac.load(
    item_collection,
    bands = ['red','scl'],
    geobox = geobox,
    chunks = {'x':1000,'y':1000},
    groupby = 'solar_day',
    resampling = 'bilinear'
)

print('Created dataset')

cloud_mask = src_dataset.scl.astype("uint16") & bitmask != 0
cloud_mask = mask_cleanup(cloud_mask) # Use default filters

print('Created mask')

src_dataset = erase_bad(src_dataset, cloud_mask)
src_dataset = src_dataset.where(src_dataset == 0)

src_dataset

In here
185
Created dataset
Created mask


<xarray.Dataset> Size: 42GB
Dimensions:      (time: 52, y: 10000, x: 10000)
Coordinates:
  * y            (y) float64 80kB 4.859e+06 4.859e+06 ... 4.759e+06 4.759e+06
  * x            (x) float64 80kB -1.408e+04 -1.408e+04 ... 8.59e+04 8.59e+04
    spatial_ref  int32 4B 25830
  * time         (time) datetime64[ns] 416B 2018-04-03T11:34:52.260000 ... 20...
Data variables:
    red          (time, y, x) float32 21GB dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>
    scl          (time, y, x) float32 21GB dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>

In [13]:
src_dataset['red'].median(dim='time',skipna=True).compute().fillna(0).astype('uint16')


/home/dibepa/.conda/envs/sentinel.mosaics/lib/python3.12/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/dibepa/.conda/envs/sentinel.mosaics/lib/python3.12/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/dibepa/.conda/envs/sentinel.mosaics/lib/python3.12/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/dibepa/.conda/envs/sentinel.mosaics/lib/python3.12/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/home/dibepa/.conda/envs/sentinel.mosaics/lib/python3.12/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransfor

<xarray.Dataset> Size: 400MB
Dimensions:      (y: 10000, x: 10000)
Coordinates:
  * y            (y) float64 80kB 4.859e+06 4.859e+06 ... 4.759e+06 4.759e+06
  * x            (x) float64 80kB -1.408e+04 -1.408e+04 ... 8.59e+04 8.59e+04
    spatial_ref  int32 4B 25830
Data variables:
    red          (y, x) uint16 200MB 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    scl          (y, x) uint16 200MB 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0